# Caso 1 Peluquera

* Pregunta 1

Programa que simula una linea de espera
una peluqueria tiene un peluquero  que se demora entre 15 y 30 minutos 
por corte. La peluqueria recibe un promedio 3 clientes por hora (es decir 
. uno 20 minutos). Se desea simular las llegadas  y servicios de 5 clientes

In [41]:
import random
import simpy
import math
num_peluqueros=1
tiempo_corte_min=15
tiempo_corte_max=30
tiempo_llegada_minutos=20
total_clientes=5
semilla=30
te=0.0 #timpo de estima total
dt=0.0 #duracion de servicio
fin=0.0 #minuto que finaliza

#primer procedimiento

def cortar(cliente):
  global dt
  R=random.random()
  tiempo=tiempo_corte_max - tiempo_corte_min
  tiempo_corte= tiempo_corte_min + (tiempo*R) #distribucion uniforme
  yield env.timeout(tiempo_corte)
  print("\o/ Corte listo a %s en %.2f minutos" %(cliente,tiempo_corte))
  dt=dt + tiempo_corte #Acumular los tiempos de usos de la simulacion


def cliente(env,name,personal):
  global te
  global fin
  llega=env.now #guardar el minuto de llegada del cliente
  print("--> %s llego a la peluqueria en el minuto %2.f" %(name,llega))
  with personal.request() as request: #espera turno
    yield request #obtener turno
    pasa=env.now #guarda el minuto cuando comienza a ser atendido
    espera=pasa-llega
    te=te+espera #acumulado del tiempo de espera
    print("%s Pasa con peluquero en el minuto %.2f habiendo esperado %.2f" %(name,pasa,espera))
    yield env.process(cortar(name)) #llamamos a l funcion cortar
    deja=env.now #guarda el minuto en que termina el proceso
    print("<--%s deja la peluqueria en minuto %.2f" %(name,deja))
    fin=deja #conserva ultimo minuto de la simulacion

def principal(env,personal):
  llegada=0
  i=0
  for i in range(total_clientes):
    R=random.random()
    llegada=-tiempo_llegada_minutos*math.log(R) #sigue una distribucion expotencial
    yield env.timeout(llegada) #dejo transcurrir un tiempo entre un cliente y otro
    i+=1
    env.process(cliente(env,'Cliente %d' % i ,personal))

In [42]:
#Simulacion peluqueria 
print("Simulacion peluqueria")
random.seed(semilla)
env=simpy.Environment() #creo el entorno de simulacion
personal=simpy.Resource(env,num_peluqueros) #crea los recursos  peluqueros
env.process(principal(env,personal))
env.run() #iniciar simulacion


Simulacion peluqueria
--> Cliente 1 llego a la peluqueria en el minuto 12
Cliente 1 Pasa con peluquero en el minuto 12.36 habiendo esperado 0.00
\o/ Corte listo a Cliente 1 en 15.45 minutos
<--Cliente 1 deja la peluqueria en minuto 27.81
--> Cliente 2 llego a la peluqueria en el minuto 37
Cliente 2 Pasa con peluquero en el minuto 37.17 habiendo esperado 0.00
--> Cliente 3 llego a la peluqueria en el minuto 46
\o/ Corte listo a Cliente 2 en 18.15 minutos
<--Cliente 2 deja la peluqueria en minuto 55.32
Cliente 3 Pasa con peluquero en el minuto 55.32 habiendo esperado 9.65
--> Cliente 4 llego a la peluqueria en el minuto 73
\o/ Corte listo a Cliente 3 en 20.96 minutos
<--Cliente 3 deja la peluqueria en minuto 76.28
Cliente 4 Pasa con peluquero en el minuto 76.28 habiendo esperado 3.45
--> Cliente 5 llego a la peluqueria en el minuto 82
\o/ Corte listo a Cliente 4 en 29.83 minutos
<--Cliente 4 deja la peluqueria en minuto 106.11
Cliente 5 Pasa con peluquero en el minuto 106.11 habiendo esp

In [43]:

#salidas
print("Indicadores obtenidos")
print("")
lpc=te/fin #logintud promedio de la cola
print("La longitud promedio de la cola: %.2f" % lpc)
tep=te/total_clientes
print("TIempo de espera promedio: %.2f" % tep)
upi=(dt/fin)/num_peluqueros
print("Uso promedio de la instalacion: %.2f" % upi)

Indicadores obtenidos

La longitud promedio de la cola: 0.29
TIempo de espera promedio: 7.50
Uso promedio de la instalacion: 0.83
